In [14]:
import numpy as np
import pandas as pd
import time
import pyodbc

In [15]:
taglistfile = 'C:\\Users\meclijda\Documents\TagList.csv'
dfTags = pd.read_csv(taglistfile,";",encoding = "ISO-8859-1")
tagList = dfTags['Tag'].tolist()
print(str(len(tagList))+ ' tags in the taglist.')
#print(dfTags['Tag'].head())
#print(list(set([x for x in tagList if tagList.count(x) > 1])))

63 tags in the taglist.


In [16]:
# read IP21 tag data
strAsOfDate = '01-SEP-15 00:00:00.0'
#strAsOfDate = '01-JAN-14 00:00:00.0'
#strAsOfDate = '24-AUG-15 11:40:00.0'
strUpToDate = '08-FEB-17 00:00:00.0'
pattern = '%d-%b-%y %H:%M:%S.0'
#epochstart = int(time.mktime(time.strptime(strAsOfDate, pattern)))

start_time = time.time()

#strConnect = 'DSN=PO_ODBC;MAXROWS=1000000'
strConnect = "Driver={AspenTech SQLplus};ADS=PO;TIMEOUT=;MAXROWS=10000000;TABLE=;CHARINT=Y;"\
    "CHARFLOAT=N;CHARTIME=N;READONLY=Y;ALLFIELDS=N;ROWID=N;CONVERTERRORS=Y;CHARISNULL=Y;TIBCO=N;"

cnn = pyodbc.connect(strConnect)

#print(tagList[0])
rng = pd.date_range(strAsOfDate,strUpToDate,freq='5min')
df = pd.DataFrame(index=rng)
#print(df.head())
for i, strTagName in enumerate(tagList):
    ts = time.time()
    print('{0:2}'.format(i) + ': getting data from tag ' + strTagName + ' ...')
    sql = "select TS_START, AVG from aggregates "\
        "where name = '" + strTagName + "' "\
        "and TS between '" + strAsOfDate + "' and '" + strUpToDate + "' "\
        "and STATUS < 2 "\
        "and period = 0:05"
    #print(sql)
    data1 = pd.read_sql(sql, cnn)
    data1['TS_START'] = pd.to_datetime(data1['TS_START'],utc=True)
    #data1['TS_START'] = data1['TS_START'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
    data1.set_index('TS_START',inplace=True)
    data1.columns = [strTagName]
    data1 = data1[data1.index.duplicated(keep='first')==False]
    #print(data1.head())
    #data1.to_csv(strTagName.replace(":","_") + '.csv')
    #data2 = data1.resample('1H', how='mean')
    #print(str(data1.count())+" - "+str(data1.mean()))
    #df = df.join(data1,how='outer')
    df = pd.concat([df, data1], axis=1)
    print('    data loaded in %4.1f seconds' % (time.time()-ts))

cnn.close
elapsed_time = time.time() - start_time
print('Total elapsed time: %5.1f seconds.' % elapsed_time)
#print(df.describe())

 0: getting data from tag POARFQCA-17901:me ...
    data loaded in 58.0 seconds
 1: getting data from tag POARAIA-17903_1a:av ...
    data loaded in 58.2 seconds
 2: getting data from tag POARAIA-17903_3a:av ...
    data loaded in 55.2 seconds
 3: getting data from tag POARTC-17902:me ...
    data loaded in 56.5 seconds
 4: getting data from tag POARAIA-17903_2ab:av ...
    data loaded in 27.1 seconds
 5: getting data from tag POARTI-17912:av ...
    data loaded in 56.6 seconds
 6: getting data from tag POARFCA-17907:me ...
    data loaded in 58.8 seconds
 7: getting data from tag POARTC-17923:me ...
    data loaded in 58.5 seconds
 8: getting data from tag POARTCA-17931:me ...
    data loaded in 55.5 seconds
 9: getting data from tag POARFIA-17904:av ...
    data loaded in 55.1 seconds
10: getting data from tag POARTCA-17922:me ...
    data loaded in 57.6 seconds
11: getting data from tag POARFC-17902:me ...
    data loaded in 58.0 seconds
12: getting data from tag POARTI-17947:av ...

In [17]:
df.describe()

C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,151475.000000,148627.000000,148625.000000,151468.000000,71651.000000,151475.000000,151237.000000,151468.000000,151475.000000,148723.000000,...,151475.000000,151237.000000,151468.000000,151015.000000,151015.000000,151475.000000,151475.000000,151475.000000,151475.000000,151475.000000
mean,23.871761,71.513382,68.307402,95.129925,97.860683,103.526511,3.097314,82.462804,198.515691,135.651187,...,58.998765,17.342544,-63.082730,1.346455,1.504830,205.210986,182.843718,183.869345,165.831276,16.716655
std,2.633070,2.512282,3.955745,1.339313,0.622579,1.198270,0.168848,2.158705,1.558089,12.468402,...,2.861413,2.039146,2.505006,0.274446,0.576327,1.653907,1.438805,1.554900,2.038795,1.932630
min,3.468945,48.740811,12.430414,91.591708,72.010378,96.731805,0.909646,39.541924,175.958551,80.193707,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.810322,...,157.287056,23.000000,-50.738568,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [18]:
start_time = time.time()
df.to_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total writing time to pickle: %5.1f seconds.' % elapsed_time)

Total writing time to pickle:   1.4 seconds.


In [19]:
start_time = time.time()
df1 = pd.read_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total loading time from pickle: %5.1f seconds.' % elapsed_time)
df1.describe()

Total loading time from pickle:   0.2 seconds.


C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,151475.000000,148627.000000,148625.000000,151468.000000,71651.000000,151475.000000,151237.000000,151468.000000,151475.000000,148723.000000,...,151475.000000,151237.000000,151468.000000,151015.000000,151015.000000,151475.000000,151475.000000,151475.000000,151475.000000,151475.000000
mean,23.871761,71.513382,68.307402,95.129925,97.860683,103.526511,3.097314,82.462804,198.515691,135.651187,...,58.998765,17.342544,-63.082730,1.346455,1.504830,205.210986,182.843718,183.869345,165.831276,16.716655
std,2.633070,2.512282,3.955745,1.339313,0.622579,1.198270,0.168848,2.158705,1.558089,12.468402,...,2.861413,2.039146,2.505006,0.274446,0.576327,1.653907,1.438805,1.554900,2.038795,1.932630
min,3.468945,48.740811,12.430414,91.591708,72.010378,96.731805,0.909646,39.541924,175.958551,80.193707,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.810322,...,157.287056,23.000000,-50.738568,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [20]:
start_time = time.time()
df.to_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total writing time to hdf5: %5.1f seconds.' % elapsed_time)

Total writing time to hdf5:   1.1 seconds.


In [21]:
start_time = time.time()
df2 = pd.read_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total loading time from hdf5: %5.1f seconds.' % elapsed_time)
df2.describe()

Total loading time from hdf5:   0.1 seconds.


C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,151475.000000,148627.000000,148625.000000,151468.000000,71651.000000,151475.000000,151237.000000,151468.000000,151475.000000,148723.000000,...,151475.000000,151237.000000,151468.000000,151015.000000,151015.000000,151475.000000,151475.000000,151475.000000,151475.000000,151475.000000
mean,23.871761,71.513382,68.307402,95.129925,97.860683,103.526511,3.097314,82.462804,198.515691,135.651187,...,58.998765,17.342544,-63.082730,1.346455,1.504830,205.210986,182.843718,183.869345,165.831276,16.716655
std,2.633070,2.512282,3.955745,1.339313,0.622579,1.198270,0.168848,2.158705,1.558089,12.468402,...,2.861413,2.039146,2.505006,0.274446,0.576327,1.653907,1.438805,1.554900,2.038795,1.932630
min,3.468945,48.740811,12.430414,91.591708,72.010378,96.731805,0.909646,39.541924,175.958551,80.193707,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.810322,...,157.287056,23.000000,-50.738568,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [22]:
start_time = time.time()
df.to_csv('rawdata.csv')
elapsed_time = time.time() - start_time
print('Total writing time to csv: %5.1f seconds.' % elapsed_time)

Total writing time to csv:  44.6 seconds.
